<h2 align="center"> Machine Learning -- Writing a Decision Tree Classifier</h2>  

***

<div class="alert alert-block alert-success">
This notebook is completed entirely in Python.<br>
    Farnam Adelkhani -- Last update: April 2nd, 2019
</div>

<h2 align="center">Statistics Review for Decision Trees</h2>  

<center><b>Decision Trees:</b> Start at root node, move to internal nodes, lastly end at leaf nodes.</center>

### Two Main Types of Decision Trees Exist:
> **Classification Trees** :  
When the predicted outcome is the class to which the data belongs.  
**Regression Trees** :  
When the predicted outcome can be considered a real number.  
(ie: Price of a car, or wait time at a restaurant).  
    
<center><b>(CART)</b> - Umbrella term for both Classification And Regression Trees. </center>

**Ensemble Learner Methods:**
> A machine learning technique that combines several base models in order to produce one optimal predictive model.  
(ie: Take a collection of 'ok' predictors and combine them into something more powerful.  

**Boosting:** Idea is to combine several weak algorithms into a strong one. Try predictors sequentially, such that each subsequent model attempts to fix the errors of its predecessor.  
> 1. Gradient Boosted Trees:  
    - Contrary to AdaBoost, which tweaks the instance weights at every interaction, this method tries to fit the new predictor to the residual errors made by the previous predictor.  
2. AdaBoost (Adaptive Boost):
    - Creating a forest of stumps (ie: 1 Node and 2 leaves)
3. Bootstrap Aggregated (Bagging): 
    - Combines bootstrapping and aggregation to form one ensemble model.
4. Rotation Forest:
    - Rotation Forest is an ensemble classifier using many decision tree models and it can be used for classification or Regression.

Decision Tree Classifier:
- Add a root node for the tree
- All nodes receive a list of rows as input.
- The root will receive the entire training set.
- Each node asks a true/false question about one of the features.
- In response split the data into 2 subsets/child nodes.  

The key to building a successful tree, is to know what questions to ask and when.  
- ie: Quantify how much a Question helps to unmix the labels.  
... Can do this with the metric: Gini Impurity

Quantify how much a Question reduces an uncertainty.  
- Can do this with the metric: Information Gain

## Gini Coefficient:
<h4 align="center">A measure of statistical dispersion intended to represent the income or wealth distribution of a nation's residents,<br> and is the most commonly used measurement of inequality.</h4> 
<img src="Decision_Tree_Classifier--Images/lorenz.png" alt="Drawing" style="width: 400px;"/>

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [2]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first three columns are features.
#
# Note: The 4th and 10th examples have the same features, but different labels...
#   such that we can see how the tree handles this case.
#
# ["feature", "feature", "feature", "label"]

training_data = [
    ['V8', 'Green', 2018, 'Audi'],
    ['EV', 'Black', 2015, 'Benz'],
    ['I4', 'White', 2018, 'BMW'],
    ['EV', 'Yellow', 2013, 'Ford'], # Same features
    ['EV', 'Blue', 2015, 'Honda'],
    ['V6', 'White', 2013, 'Honda'],
    ['V8', 'Green', 2016, 'Lexus'],
    ['V6', 'Red', 2015, 'Lexus'],
    ['V8', 'White', 2018, 'Lexus'],
    ['EV', 'Yellow', 2013, 'Tesla'], # Same features
]

In [3]:
# Column labels:
header = ["engine", "color", "year", "label"]

In [4]:
# Find the unique values for a column in a dataset (unsorted):
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [5]:
# Demo of unique_vals function for all column:
engine = unique_vals(training_data, 0)
color = unique_vals(training_data, 1)
year = unique_vals(training_data, 2)
make = unique_vals(training_data, 3)

print(engine, color, year, make, sep='\n')

{'EV', 'I4', 'V8', 'V6'}
{'Black', 'White', 'Red', 'Green', 'Yellow', 'Blue'}
{2016, 2018, 2013, 2015}
{'Ford', 'Tesla', 'Lexus', 'Honda', 'Benz', 'Audi', 'BMW'}


In [6]:
def class_counts(rows):
    """Count # of each type of label in the dataset."""
    # Dictionary of labels
    counts = {}
    for row in rows:
        # The label is usually the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
# Demo of class_counts function:
class_counts(training_data)

{'Audi': 1, 'Benz': 1, 'BMW': 1, 'Ford': 1, 'Honda': 2, 'Lexus': 3, 'Tesla': 1}

In [8]:
def is_numeric(value):
    """Test if value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [9]:
# Demo of is_numeric function:
digit = is_numeric(7) # Expected: True
color = is_numeric("Red") # Expected: False

print(digit, color, sep='\n')

True
False


In [10]:
class Question:
    """Question class is used to partition a dataset.

    This class just records a 'column number'
    - (ex: 0 for Engine) and a 'column value' (ex: V6 or EV). 
    The 'match' method is used to compare the feature value in an example 
    to the feature value stored in the question.
    """

    def __init__(self, column, value):
        # Represent a question by storing a column # and value 
        # ... for threshhold used to partition the data
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # ... feature value in this question.
        val = example[self.column]
        # Check if val is numerical:
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # ... the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [11]:
# Demo of Question class:
# Let's write a question for a numeric attribute, is year >= 2015?
Question(2, 2015)

Is year >= 2015?

In [12]:
# Demo of Question class:
# Ask if the color is green
q = Question(1, 'Green')
q

Is color == Green?

In [13]:
# Let's pick an example from the training set...
# 0 refers to the 1st row, not column
example = training_data[0]
# print(example)
# ... and see if it matches the question
q.match(example) # this will be true, since the first row contains "Green"

True

In [14]:
def partition(rows, question):
    """Partitions a dataset by row.

    For each row in the dataset, check if it matches the question. 
    If so:
        add it to 'true rows'
    else: 
        add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [15]:
# Demo of partition function:
# Partition the training data based on whether rows contain Green.
true_rows, false_rows = partition(training_data, Question(1, 'Green'))

# This will contain all the 'Green' rows.
true_rows
# This will contain everything else.
# false_rows

[['V8', 'Green', 2018, 'Audi'], ['V8', 'Green', 2016, 'Lexus']]

In [16]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
       ie: How much uncertainty there is at a node.

    There are different means of acheiving this, this is one way:
    Wikipedia Link:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [17]:
# Demo to understand how Gini Impurity works:

# First, a dataset with no mixing.
no_mixing = [['Ford'],
             ['Ford']] #This should return 0

# Now, a dataset with a 50:50 Honda:Lexus ratio
mixing = [['Honda'],
          ['Lexus']] #This should return 0.5

# No mixing
print(gini(no_mixing))

# Mixing
print(gini(mixing)) 
   # 'mixing' will return 0.5, meaning, there's a 50% chance of misclassifying a random example we draw from the dataset.

0.0
0.5


<img src="Decision_Tree_Classifier--Images/gini_impurity.png" alt="Drawing" style="width: 450px;"/>
<h3 align="center">ie: We have a 1/5 chance of being correct if randomly assign an object to a label.</h3> 

In [18]:
# Looking at the dataset pictured above:
lots_of_mixing = [['Apple'],
                  ['Banana'],
                  ['Grape'],
                  ['Lemon'],
                  ['Orange'],]

# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [19]:
def info_gain(left, right, current_uncertainty):
    """Information Gain: Find the question that reduces uncertainty the most.

    uncertainty of the starting node - the weighted impurity of two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [20]:
# Demo:
# Calculate the uncertainy of training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.82

In [21]:
# How much information do we gain by partioning on 'Yellow'?
true_rows, false_rows = partition(training_data, Question(1, 'Yellow'))
info_gain(true_rows, false_rows, current_uncertainty)

0.11999999999999988

In [22]:
# What about if we partioned on 'White' instead?
true_rows, false_rows = partition(training_data, Question(1,'White'))
info_gain(true_rows, false_rows, current_uncertainty)

0.04857142857142871

In [23]:
# It looks like we learned more using 'Yellow' (0.119), than 'White' (0.048).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(1,'Yellow'))

# Here, the true_rows contain only 'Tesla'.
true_rows

[['EV', 'Yellow', 2013, 'Ford'], ['EV', 'Yellow', 2013, 'Tesla']]

In [24]:
# And the false rows contain multiple vehicle manufacturers:
false_rows

[['V8', 'Green', 2018, 'Audi'],
 ['EV', 'Black', 2015, 'Benz'],
 ['I4', 'White', 2018, 'BMW'],
 ['EV', 'Blue', 2015, 'Honda'],
 ['V6', 'White', 2013, 'Honda'],
 ['V8', 'Green', 2016, 'Lexus'],
 ['V6', 'Red', 2015, 'Lexus'],
 ['V8', 'White', 2018, 'Lexus']]

In [25]:
# On the other hand, partitioning by Blue doesn't help so much.
true_rows, false_rows = partition(training_data, Question(1,'Blue'))

# We've isolated one 'Honda' in the true rows.
true_rows

[['EV', 'Blue', 2015, 'Honda']]

In [26]:
# But, the false-rows are badly mixed up.
false_rows

[['V8', 'Green', 2018, 'Audi'],
 ['EV', 'Black', 2015, 'Benz'],
 ['I4', 'White', 2018, 'BMW'],
 ['EV', 'Yellow', 2013, 'Ford'],
 ['V6', 'White', 2013, 'Honda'],
 ['V8', 'Green', 2016, 'Lexus'],
 ['V6', 'Red', 2015, 'Lexus'],
 ['V8', 'White', 2018, 'Lexus'],
 ['EV', 'Yellow', 2013, 'Tesla']]

In [27]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    # Iterate over every value for the features:
    for col in range(n_features):

        # unique values in the column
        values = set([row[col] for row in rows])

        # for each value
        for val in values:

            # Generate a question for that feature:
            question = Question(col, val)

            # Try splitting the dataset, ie: partition the data on it:
            true_rows, false_rows = partition(rows, question)

            # Discard any questions that fail to produce a split:
            # Take a weighted average of the impurity of each set:
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [28]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is engine == EV?

In [29]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [30]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [31]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    # Find the best question to ask at this node:
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [32]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [33]:
# When calling 'build_tree' it receives entire training tree as input
# my_tree = Returns ref to root node of tree:
my_tree = build_tree(training_data)

In [34]:
print_tree(my_tree)

Is engine == EV?
--> True:
  Is color == Black?
  --> True:
    Predict {'Benz': 1}
  --> False:
    Is color == Yellow?
    --> True:
      Predict {'Ford': 1, 'Tesla': 1}
    --> False:
      Predict {'Honda': 1}
--> False:
  Is year >= 2015?
  --> True:
    Is engine == I4?
    --> True:
      Predict {'BMW': 1}
    --> False:
      Is year >= 2018?
      --> True:
        Is color == Green?
        --> True:
          Predict {'Audi': 1}
        --> False:
          Predict {'Lexus': 1}
      --> False:
        Predict {'Lexus': 2}
  --> False:
    Predict {'Honda': 1}


In [35]:
## To Do: Visualizing the decision tree above:
# Load libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from IPython.display import Image  
from sklearn import tree
import pydotplus

# Create decision tree classifer object
clf = DecisionTreeClassifier(random_state=0)



In [36]:
# Classify data, starts with reference to root node:

def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [37]:
# Demo:
# The tree predicts the 1st row of our
# training data is an Audi with confidence 1.
classify(training_data[0], my_tree)

{'Audi': 1}

In [38]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [39]:
# Print utilizing a percentage:
print_leaf(classify(training_data[0], my_tree))

{'Audi': '100%'}

In [40]:
# Demo:
# On the 4th row, the confidence is lower
# This is because the same features are shared with another label
print_leaf(classify(training_data[3], my_tree))

{'Ford': '50%', 'Tesla': '50%'}

In [41]:
# Evaluate
testing_data = [
    ['V8', 'Green', 2018, 'Audi'],
    ['EV', 'Black', 2015, 'Benz'],
    ['I4', 'White', 2018, 'BMW'],
    ['EV', 'Yellow', 2013, 'Ford'], # Same features
    ['EV', 'Blue', 2015, 'Honda'],
    ['V6', 'White', 2013, 'Honda'],
    ['V8', 'Green', 2016, 'Lexus'],
    ['V6', 'Red', 2015, 'Lexus'],
    ['V8', 'White', 2018, 'Lexus'],
    ['EV', 'Yellow', 2013, 'Tesla'], # Same features
]

In [42]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Audi. Predicted: {'Audi': '100%'}
Actual: Benz. Predicted: {'Benz': '100%'}
Actual: BMW. Predicted: {'BMW': '100%'}
Actual: Ford. Predicted: {'Ford': '50%', 'Tesla': '50%'}
Actual: Honda. Predicted: {'Honda': '100%'}
Actual: Honda. Predicted: {'Honda': '100%'}
Actual: Lexus. Predicted: {'Lexus': '100%'}
Actual: Lexus. Predicted: {'Lexus': '100%'}
Actual: Lexus. Predicted: {'Lexus': '100%'}
Actual: Tesla. Predicted: {'Ford': '50%', 'Tesla': '50%'}


<div class="alert alert-block alert-info">
Credits:
Josh Gordon @ Google: https://github.com/random-forests/tutorials
</div>